### Theme1

#### Download Dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sanikamal/rock-paper-scissors-dataset")

print("Path to dataset files:", path)

100%|██████████| 452M/452M [00:02<00:00, 226MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/sanikamal/rock-paper-scissors-dataset/versions/1


In [3]:
!mv /root/.cache/kagglehub/datasets/sanikamal/rock-paper-scissors-dataset/versions/1 /content

In [4]:
!mv 1/Rock-Paper-Scissors/ /content/Rock-Paper-Scissors/

#### EDA

In [5]:
!ls

1  Rock-Paper-Scissors	sample_data


In [6]:
import os

path = "/content/Rock-Paper-Scissors"

for root, dirs, files in os.walk(path):
    for d in dirs:
        print(os.path.join(root, d))

/content/Rock-Paper-Scissors/train
/content/Rock-Paper-Scissors/test
/content/Rock-Paper-Scissors/validation
/content/Rock-Paper-Scissors/train/rock
/content/Rock-Paper-Scissors/train/paper
/content/Rock-Paper-Scissors/train/scissors
/content/Rock-Paper-Scissors/test/rock
/content/Rock-Paper-Scissors/test/paper
/content/Rock-Paper-Scissors/test/scissors


In [7]:
from pathlib import Path
import json

root = Path(path)  
splits = ["train", "validation", "test"]
summary = {}

for split in splits:
    split_dir = root / split
    summary[split] = {}
    if split_dir.is_dir():
        if any(child.is_dir() for child in split_dir.iterdir()):
            # train/test
            for cls_dir in sorted(split_dir.iterdir()):
                if cls_dir.is_dir():
                    summary[split][cls_dir.name] = len(list(cls_dir.glob("*.png")))
        else:
            # validation 
            summary[split] = {"rock": 0, "paper": 0, "scissors": 0}
            for img_path in split_dir.glob("*.png"):
                name = img_path.name.lower()
                for cls in summary[split]:
                    if cls in name:
                        summary[split][cls] += 1
                        break
    summary[split]["total"] = sum(summary[split].values())

summary["overall_total"] = sum(summary[s]["total"] for s in splits)

with open("data_summary.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(json.dumps(summary, indent=2, ensure_ascii=False))


{
  "train": {
    "paper": 840,
    "rock": 840,
    "scissors": 840,
    "total": 2520
  },
  "validation": {
    "rock": 11,
    "paper": 11,
    "scissors": 11,
    "total": 33
  },
  "test": {
    "paper": 124,
    "rock": 124,
    "scissors": 124,
    "total": 372
  },
  "overall_total": 2925
}


#### Clean Dataset

In [15]:
import tensorflow as tf
import numpy as np
from pathlib import Path

SEED = 77

PATH = "/content/Rock-Paper-Scissors"
ROOT = Path(path)


IMG_SIZE = (300, 300)
BATCH_SIZE = 4

train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    ROOT /"train",
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    root / "test",
    labels="inferred",
    label_mode="int",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

Found 2520 files belonging to 3 classes.
Found 372 files belonging to 3 classes.


In [16]:
label_map = {"rock": 0, "paper": 1, "scissors": 2}
val_files = sorted((root / "validation").glob("*.png"))

def parse_val(path_str):
    path = Path(path_str.numpy().decode())
    label = next(v for k, v in label_map.items() if k in path.name.lower())
    img = tf.keras.utils.load_img(path, target_size=IMG_SIZE)
    arr = tf.keras.utils.img_to_array(img)
    return arr, label

def tf_parse_val(path):
    img, label = tf.py_function(parse_val, [path], [tf.float32, tf.int64])
    img.set_shape((*IMG_SIZE, 3))
    label.set_shape(())
    return img, label

val_ds_raw = tf.data.Dataset.from_tensor_slices([str(p) for p in val_files]) \
    .map(tf_parse_val, num_parallel_calls=tf.data.AUTOTUNE) \
    .batch(BATCH_SIZE)

#### Augumentation

In [17]:
import tensorflow as tf

# 1) 前處理層：resize + rescale
base_preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(*IMG_SIZE),
    tf.keras.layers.Rescaling(1./255),
], name="base_preprocess")

# 2) 資料增強：僅訓練用
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.15),
    tf.keras.layers.RandomBrightness(factor=0.1),
], name="data_augmentation")

def preprocess_dataset(ds, training=False):
    ds = ds.map(lambda x, y: (base_preprocess(x), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.shuffle(512)
    return ds.prefetch(tf.data.AUTOTUNE)

train_ds = preprocess_dataset(train_ds_raw, training=True)
val_ds   = preprocess_dataset(val_ds_raw)
test_ds  = preprocess_dataset(test_ds_raw)


#### Model Building

In [24]:
def build_scratch_model(input_shape=(300, 300,3), num_classes=3):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(24, 3, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.MaxPool2D()(x)

    for filters in [64, 128, 256]:
        x = tf.keras.layers.Conv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.Conv2D(filters, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        x = tf.keras.layers.MaxPool2D()(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    return tf.keras.Model(inputs, outputs)


#### Training

In [25]:
model = build_scratch_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("scratch_best.keras",
                                       monitor="val_accuracy",
                                       save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=3, min_lr=1e-5),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=6, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=callbacks
)


Epoch 1/40


KeyboardInterrupt: 

#### Plot

In [ ]:


plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss"); plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy"); plt.legend()
plt.show()


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test acc: {test_acc:.3f}")

y_true, y_pred = [], []
for imgs, labels in test_ds:
    probs = model.predict(imgs, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(probs, axis=1))

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_map.keys(),
            yticklabels=label_map.keys())
plt.xlabel("Predicted"); plt.ylabel("True"); plt.show()
print(classification_report(y_true, y_pred, target_names=list(label_map.keys())))
